In [467]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
import datetime as dt
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data


In [577]:
'''
Load  bag filesto make into respective dataframes
'''
defaultPath="/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2017_03_31/"
# defaultPath="/home/rhagoletis/catkin/src/World/bags/"

paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*traj.bag"])

# paths =['/media/rhagoletis/NICE_Pavan_2B/2017_04_17/2017-04-17__23~05~43_apple34_00_odour_gain~8_speed~1.0_bout~30_DC~-0.128_traj.bag'] 
print paths, "\n"
metadata=None
# t=time.now()
i=1
for path in paths:
    tic()
    print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    bag=rosbag.Bag(path)
    
    try:
        for topic,msg,t in bag.read_messages(topics='/metadata'):
            a=msg
#         parameters=json.loads(a.data)
#         metadata={"meta":parameters}

        metadata=json.loads(a.data)

    except :
        print "no such file!, trying the pickle"
        try:
            metadata=depickler(paths[0].split('.bag')[0])
        except IOError:
            metadata=depickler(paths[0].split('.bag')[0]+'.pickle')
            
        parameters=metadata['parameters']
        
            
    obj=dict(df=df,metadata=metadata)
    
    picklepath=path+"_df.pickle"
    pickler(obj,picklepath)
#     df.to_pickle(picklepath)
    
    i+=1
    toc()
print "\nanalysis of %s files complete" %len(paths)
# print (time.now()-t)

['/home/rhagoletis/catkin/src/World/bags/2017_06_06/2017-06-06__19~51~30_haw09_01_gain~8_speed~1.0_bout~5_DC~-0.002_traj.bag'] 

starting analysis of file 2017-06-06__19~51~30_haw09_01_gain~8_speed~1.0_bout~5_DC~-0.002_traj.bag , 1 / 1 files
Elapsed time: 4.792155 seconds.


analysis of 1 files complete


In [471]:
t=dt.datetime.now()
time.sleep(3)
print dt.datetime.now()-t



0:00:03.015371


In [ ]:
pick=depickler(paths[0].split('.bag')[0])


In [ ]:
df.to_csv('f.csv')

In [3]:
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np

win = pg.GraphicsWindow()
win.resize(800,350)
win.setWindowTitle('pyqtgraph example: Histogram')
plt1 = win.addPlot()

## make interesting distribution of values
vals = np.hstack([np.random.normal(size=500), np.random.normal(size=260, loc=4)])

## compute standard histogram
y,x = np.histogram(vals, bins=np.linspace(-3, 8, 40))

## notice that len(x) == len(y)+1
## We are required to use stepMode=True so that PlotCurveItem will interpret this data correctly.
curve = pg.PlotCurveItem(x, y, stepMode=True, fillLevel=0, brush=(0, 0, 255, 80))
plt1.addItem(curve)

In [ ]:
'''
Load  bag filesto make into respective dataframes
1726819
'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*traj.bag"])
print paths, "\n"
metadata=None

i=1
for path in paths:
    tic()
    print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    bag=rosbag.Bag(path)
    
#     try:
#         for topic,msg,t in bag.read_messages(topics='/metadata'):
#             a=msg
# #         parameters=json.loads(a.data)
# #         metadata={"meta":parameters}
#         metadata=json.loads(a.data)

#     except :
#         print "no such file!"
            
#     obj=dict(df=df,metadata=metadata)
    
#     picklepath=path+"_df.pickle"
#     pickler(obj,picklepath)
# #     df.to_pickle(picklepath)
    
    i+=1
    toc()
print "\nanalysis of %s files complete" %len(paths)


In [ ]:
df

In [ ]:
parameters=obj['metadata']['parameters']
print parameters

In [ ]:

df[['trajectory__object1Pos_x', 'trajectory__object1Pos_y']]

In [ ]:
# plt.scatter(df.trajectory__position_x,df.trajectory__position_y)
# plt.show()
# df.trajectory__wbad.plot()
# df.trajectory__wbas.plot()
a=(pd.rolling_mean(df.trajectory__wbas,30))
a.plot()
df.trajectory__reset.plot()
plt.show()

In [ ]:
df

In [253]:

def analyser(winTitle):
    
    '''
    takes window title input and returns average 
    compensation during stimulus period for each speed as a df
    '''
    #unknown rare bug, shabby fix
    df.trajectory__impose[0]=0
    
    impose=df.trajectory__impose.copy()
    impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
    compensation=df.trajectory__imposeResponse/impose
    df.trajectory__imposeResponseSmooth=pd.rolling_mean(df.trajectory__imposeResponse,window=165)
    compensationSmooth=df.trajectory__imposeResponseSmooth/impose

    cp = pd.DataFrame(index=compensation.index,columns=['a'])

    #flips are position where there is a rise or fall
    flips=np.sign(np.diff(df.trajectory__impose))

    #impose is impose df without first element
    impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

    #cw is clockWise, ccw is Counter clockwise
    #start and stop are positions where the impose starts or stops
    #start(stop) is zero everywhere except when there is the respective start(stop) event
    ccwStart=np.logical_and((flips==1) , (impose>0))
    ccwStop=np.logical_and((flips==-1) , (impose==0))
    cwStart=np.logical_and((flips==-1) , (impose<0))
    cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

    #(c)cwStart(stop)Index is just those positions where flips occured

    ccwStartIndex= ccwStart[ccwStart==1]
    cwStartIndex= cwStart[cwStart==1]
    ccwStopIndex= ccwStop[ccwStop==1]
    cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


    #compensationMean is a df who is zero when impose is false, 
    # and has mean value of compensation during the corresponding impose
    compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

    # print compensation
    def compMean(start,stop,compIn,compOut):
        consol=pd.DataFrame(columns=['fly','gain','impose','compensation'])


        print "calculating Mean"
        #if there are more starts than stop because of abrupt ending while recording
        #remove the last start event
        
        #remove the last events to remove noise
        start=start[:-1]
        stop=stop[:-1]

        try: 
            assert len(start)==len(stop)
        except AssertionError:
            print 'assertion error'
            start=start[:-1] 

        for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
            currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
            compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
            curr=pd.DataFrame([{'fly':parameters['fly'],
                                                 "gain":df.trajectory__gain[start.index[i]],
                                                 "impose":df.trajectory__impose[start.index[i]],
                                                 'compensation':currCompensationMean}])
            consol=consol.append(curr,ignore_index=True)        
        cols=['fly','gain','impose','compensation']
        consol=consol[cols]
#         print "total measn were",i+1
        return consol

    cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
    ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)
    
    
    
    plt.plot(df.trajectory__imposeResponseSmooth,c=(0,0,1,0.2),label='imposeResponseSmooth')
#     plt.plot(compensationSmooth,c=(1,0,0,0.6),label='compensationSmooth')
    plt.plot(df.trajectory__impose,label='impose')
    plt.plot(cp,'g',label='compensationMean')
    plt.axhline(y=1)
    plt.title(winTitle)
    plt.legend()
    plt.show()
    
    return ccw, cw


In [584]:

def analyser2(winTitle):
    
    '''
    takes window title input and returns average 
    compensation during stimulus period for each speed as a df
    '''
    #unknown rare bug, shabby fix
    df.trajectory__impose[0]=0
    
    impose=df.trajectory__impose.copy()
    impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
    imposeSign=np.sign(impose)
#     imposeSign[imposeSign==0]=1
#     compensation=(df.trajectory__imposeResponse-df.trajectory__impose)#*imposeSign
#     compensation=(df.trajectory__imposeResponse-df.trajectory__impose)*imposeSign/abs(impose)
    compensation=(df.trajectory__impose-df.trajectory__imposeResponse)


    
    df.trajectory__imposeResponseSmooth=pd.rolling_mean(df.trajectory__imposeResponse,window=10)
    compensationSmooth=df.trajectory__impose-df.trajectory__imposeResponseSmooth

    cp = pd.DataFrame(index=compensation.index,columns=['a'])

    #flips are position where there is a rise or fall
    flips=np.sign(np.diff(df.trajectory__impose))

    #impose is impose df without first element
    impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

    #cw is clockWise, ccw is Counter clockwise
    #start and stop are positions where the impose starts or stops
    #start(stop) is zero everywhere except when there is the respective start(stop) event
    ccwStart=np.logical_and((flips==1) , (impose>0))
    ccwStop=np.logical_and((flips==-1) , (impose==0))
    cwStart=np.logical_and((flips==-1) , (impose<0))
    cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

    #(c)cwStart(stop)Index is just those positions where flips occured

    ccwStartIndex= ccwStart[ccwStart==1]
    cwStartIndex= cwStart[cwStart==1]
    ccwStopIndex= ccwStop[ccwStop==1]
    cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


    #compensationMean is a df who is zero when impose is false, 
    # and has mean value of compensation during the corresponding impose
    compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

    # print compensation
    def compMean(start,stop,compIn,compOut):
        consol=pd.DataFrame(columns=['fly','gain','impose','compensation','error'])


#         print "calculating Mean"
        #if there are more starts than stop because of abrupt ending while recording
        #remove the last start event
        
        #remove the last events to remove noise
        start=start[:-1]
        stop=stop[:-1]

        try: 
            assert len(start)==len(stop)
        except AssertionError:
#             print 'assertion error'
            start=start[:-1] 

        for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
            currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
            currErrorMean=compIn.ix[start.index[i]:stop.index[i]].std()
        
            compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
            curr=pd.DataFrame([{'fly':parameters['fly'],
                                                 "gain":df.trajectory__gain[start.index[i]],
                                                 "impose":df.trajectory__impose[start.index[i]],
                                                 'compensation':currCompensationMean,"error":currErrorMean}])
            consol=consol.append(curr,ignore_index=True)        
        cols=['fly','gain','impose','compensation','error']
        consol=consol[cols]
#         print "total measn were",i+1
        return consol

    cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
    ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)
    
    
#     plt.figure()
#     plt.plot(df.trajectory__imposeResponseSmooth,c=(0,0,0.7,0.3),lw=2,label='imposeResponseSmooth')
# #     plt.plot(df.trajectory__imposeResponse,c=(0,0,1,0.2),label='imposeResponse')
#     plt.plot(compensationSmooth,c=(1,0,0,0.5),lw=1,label='compensationSmooth')
# #     plt.plot(compensation,c=(1,0,0,0.6),label='compensation')

#     plt.plot(df.trajectory__impose,label='impose')
#     plt.plot(cp,'g',label='compensationMean',lw=2,alpha=1)
# #     plt.axhline(y=1)
#     plt.title(winTitle)
#     plt.legend()
#     plt.show()
    
    return ccw, cw


In [531]:
fullConsol=pd.DataFrame()
ccw,cw=analyser2(Wtitle)
fullConsol=fullConsol.append(ccw,ignore_index=True)
fullConsol=fullConsol.append(cw,ignore_index=True)
# fullConsol

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:21: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()


In [648]:
'''Load a dataframe from pickle and analyse impose compensation'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*df.pickle"])
# paths=['/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__00:08:57_fly13_quad_00_gain3.5_trial_1_traj.bag_df.pickle'] 
# paths=['/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__00:08:57_fly13_quad_00_gain3.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__00:23:58_fly13_quad_00_gain7_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__01:03:45_fly13_quad_00_gain10.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__01:17:49_fly13_quad_00_gain14_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__01:40:40_fly13_quad_00_gain17.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__01:52:56_fly13_quad_00_gain21_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__02:10:40_fly13_quad_00_gain28_trial_1_traj.bag_df.pickle']
# paths=['/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly13/2016-09-03__00:08:57_fly13_quad_00_gain3.5_trial_1_traj.bag_df.pickle'] 
fullConsol=pd.DataFrame()
print paths, "\n"
for path in paths:
    
    tic()
    print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
#     parameters=data["metadata"]['meta']
    parameters=data["metadata"]['parameters']

    toc()

    Wtitle=path.split('/')[-1]
    print Wtitle
    
#     print "parameter keys are", parameters.keys()
    print "\nparameter keys length are", len(parameters.keys())

    df.trajectory__imposeResponse.plot(title=path.split('/')[-1],label="impose response")
    df.trajectory__impose.plot(title=path.split('/')[-1],label="impose")
    df.trajectory__compensation.plot(title=path.split('/')[-1],label="impose")

#     plt.show()
    
    ccw,cw=analyser2(Wtitle)
    fullConsol=fullConsol.append(ccw,ignore_index=True)
    fullConsol=fullConsol.append(cw,ignore_index=True)

# fullConsol["compensation"]=-fullConsol["compensation"]

# print fullConsol

['/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-04__22:45:50_fly14_impose_quad_00_gain3.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-04__23:37:01_fly14_impose_quad_00_gain7_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-04__23:53:17_fly14_impose_quad_00_gain14_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__20:54:12_fly14_impose_quad_00_gain3.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__21:34:09_fly14_impose_quad_00_gain3_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__21:36:49_fly14_impose_quad_00_gain3_trial_1_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__23:16:41_fly14_impose_quad_00_gain3.5_trial_1_traj.bag_df.pickle', '/home/rhagoletis/c

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:21: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()




currentl;y analysisnh /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-04__23:37:01_fly14_impose_quad_00_gain7_trial_1_traj.bag_df.pickle
Elapsed time: 2.843155 seconds.

2016-09-04__23:37:01_fly14_impose_quad_00_gain7_trial_1_traj.bag_df.pickle

parameter keys length are 106


currentl;y analysisnh /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-04__23:53:17_fly14_impose_quad_00_gain14_trial_1_traj.bag_df.pickle
Elapsed time: 1.820841 seconds.

2016-09-04__23:53:17_fly14_impose_quad_00_gain14_trial_1_traj.bag_df.pickle

parameter keys length are 106


currentl;y analysisnh /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__20:54:12_fly14_impose_quad_00_gain3.5_trial_1_traj.bag_df.pickle
Elapsed time: 0.469094 seconds.

2016-09-05__20:54:12_fly14_impose_quad_00_gain3.5_trial_1_traj.bag_df.pickle

parameter keys length are 106


currentl;y analysisnh /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/2016-09-05__

In [649]:
def consolidater():
    path=easygui.filesavebox()
    path+='_'+data['metadata']['parameters']['fly']+'_'+'consolidatedDF.pickle'

    print "save path is",path
    dataSet=dict(metadata=data['metadata'],fullConsol=fullConsol)
    pickler(dataSet,path)
consolidater()

save path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly14/14_fly14_consolidatedDF.pickle


In [601]:
def unConsolidater(path=''):
    if path is "":
        path=easygui.fileopenbox()

    print "file path is",path
    dataset=depickler(path)
    return dataset["metadata"], dataset["fullConsol"]
met,fullConsol=unConsolidater()

file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/fly9/9_fly9_consolidatedDF.pickle


In [461]:
import glob
print glob.glob('/home')

['/home']


In [650]:
fullConsol

,fly,gain,impose,compensation,error
0,fly14,3.5,0.25,-0.094407,0.589985
1,fly14,3.5,0.50,-0.090531,0.477979
2,fly14,3.5,1.00,0.091333,0.541617
3,fly14,3.5,2.00,0.697451,0.721469
4,fly14,3.5,4.00,2.293460,1.019007
5,fly14,3.5,8.00,7.028295,0.776628
6,fly14,3.5,8.00,7.578070,0.593580
7,fly14,3.5,4.00,2.228711,1.082816
8,fly14,3.5,2.00,0.477141,0.629332
9,fly14,3.5,1.00,0.160154,0.463322


In [660]:
paths =easygui.fileopenbox(multiple=True)
fullConsol=pd.DataFrame()
for path in paths:
    met,fc=unConsolidater(path)
    fullConsol=fullConsol.append(fc,ignore_index=True)
    print fc.shape
print "cOmplete"


file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/Consolidated/9_fly9_consolidatedDF.pickle
(487, 5)
file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/Consolidated/11_fly11_consolidatedDF.pickle
(155, 5)
file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/Consolidated/12_fly11_consolidatedDF.pickle
(175, 5)
file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/Consolidated/13_fly13_consolidatedDF.pickle
(553, 5)
file path is /home/rhagoletis/catkin/src/World/ipy_notebooks/impose/Consolidated/14_fly14_consolidatedDF.pickle
(613, 5)
cOmplete


In [661]:
fullConsol.fly.unique()

array([u'fly9', u'fly11', u'fly13', u'fly14'], dtype=object)

In [657]:
fullConsol2=fullConsol[((abs(fullConsol["impose"])>2) &(abs(fullConsol["impose"])<7) &(abs(fullConsol["gain"])>3)  )]
# fullConsol2

In [654]:
# fullConsol["compensation"]=-fullConsol["compensation"]
fullConsol["compensation"]=abs(fullConsol["compensation"])
fullConsol["impose"]=abs(fullConsol["impose"])




In [658]:
ax=sns.regplot(x=fullConsol2['gain'], y=fullConsol2['error'],x_jitter=.15,scatter_kws={"s": 28},label='Maneuverability');
ax=sns.regplot(x=fullConsol2['gain'], y=fullConsol2['compensation'],x_jitter=.15,scatter_kws={"s": 28},label='Stability',);
# ax=sns.regplot(x=fullConsol['gain'], y=fullConsol['compensation'],x_jitter=.15,scatter_kws={"s": 28},label='Stability');
sns.set(font_scale=1.25)  # crazy big
ax.legend()
ax.set(xlabel='Gain (165*deg/rad/s)', ylabel='Stability and Maneuverability (rad/s)',title='Tradeoff between Stability and Maneuverability',)
pth=path+"regplot.png"
plt.savefig(pth, dpi=400)

# sns.regplot(x=fullConsol['gain'], y=fullConsol['error'],x_jitter=.5);
# 
# sns.lmplot(x="total_bill", y="tip", hue="smoker", data=tips);

In [655]:
#lm plot of 2 graphs with impose fits

ax1=sns.lmplot(x="gain", y="error",x_jitter=.25,scatter_kws={"s": 28},hue="impose",data=fullConsol)
ax1.set(xlabel='Gain (165*deg/rad/s)', ylabel='S.D. of impose respone (rad/s)',title='Effect of impose speed on Maneuverability',)
ax2=sns.lmplot(x="gain", y="compensation",x_jitter=.25,scatter_kws={"s": 28},hue="impose",data=fullConsol)
ax2.set(xlabel='Gain (165*deg/rad/s)', ylabel='Mean of impose respone (rad/s)',title='Effect of impose speed on Stability',)
plt.tight_layout()

In [595]:
ax=sns.regplot(x="gain", y="compensation",data=fullConsol,x_jitter=.25,scatter_kws={"s": 28},label='Stability');
ax=sns.regplot(x="gain", y="error",data=fullConsol,x_jitter=.25,scatter_kws={"s": 28},label='Stability');
ax.set(xlabel='Gain (165*deg/rad/s)', ylabel='Stability and Maneuverability (rad/s)',title='Tradeoff between Stability and Maneuverability')


In [ ]:
import seaborn as sns
# sns.set_style("whitegrid")

In [388]:
plt.figure()

# ax = sns.stripplot(hue=fullConsol['impose'].round(1).abs(),   jitter=0.092,
#                    y=(1-fullConsol['compensation']),x=fullConsol['gain'],
#                    split=True,palette="viridis",linewidth=1)
ax = sns.stripplot( jitter=0.092,hue=fullConsol['impose'].round(1).abs(),
                   y=(abs(fullConsol['compensation'])),x=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=1)
for val in np.unique(abs(fullConsol["impose"])):
    plt.axhline(val)

plt.show()

In [386]:
#yay, 
plt.figure()
#hue=fullConsol['impose'].round(1).abs(),  
ax = sns.stripplot( jitter=0.2,
                   y=fullConsol['error'],x=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=.5,alpha=0.5)
plt.axhline(y=1)

plt.show()

In [383]:
plt.figure()

ax = sns.stripplot(x=fullConsol['impose'].round(1).abs(),   jitter=True,
                   y=fullConsol['compensation'],hue=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=0.5,alpha=0.5)
plt.axhline(y=1)

plt.show()

In [73]:
ax = sns.stripplot(x=fullConsol['impose'].round(1).abs(),   jitter=True,
                   y=fullConsol['error'],hue=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=1)
plt.axhline(y=1)

plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.violinplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            hue=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis")
plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.violinplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            x=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis")
plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.boxplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            x=fullConsol["gain"], palette="viridis")
# sns.stripplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             x=fullConsol["gain"], palette="viridis_r",jitter=True,split=True)
# plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.boxplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            hue=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis_r",jitter=True,split=True)
# plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
pickler(fullConsol,'consolidatedDF')

In [ ]:
a=depickler('consolidatedDF')

In [ ]:
a['impose']=a['impose'].abs()
b=a.groupby(['fly','gain','impose'],as_index=False).mean()

ax = sns.stripplot(hue=b['impose'].round(1).abs(),   jitter=False,
                   y=b['compensation'],x=b['gain'],split=False,palette="viridis_r",size=8)
plt.show()

In [ ]:
plt.plot(b['gain'],b['compensation'])
plt.show()

In [ ]:
b['gain']
b['compensation']

In [ ]:


'''
takes window title input and returns average 
compensation during stimulus period for each speed as a df
'''
#unknown rare bug, shabby fix
df.trajectory__impose[0]=0

impose=df.trajectory__impose.copy()
impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
# compensation=df.trajectory__imposeResponse/impose
compensation=
compensationSmooth=df.trajectory__imposeResponseSmooth/impose

cp = pd.DataFrame(index=compensation.index,columns=['a'])

#flips are position where there is a rise or fall
flips=np.sign(np.diff(df.trajectory__impose))

#impose is impose df without first element
impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

#cw is clockWise, ccw is Counter clockwise
#start and stop are positions where the impose starts or stops
#start(stop) is zero everywhere except when there is the respective start(stop) event
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

#(c)cwStart(stop)Index is just those positions where flips occured

ccwStartIndex= ccwStart[ccwStart==1]
cwStartIndex= cwStart[cwStart==1]
ccwStopIndex= ccwStop[ccwStop==1]
cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


#compensationMean is a df who is zero when impose is false, 
# and has mean value of compensation during the corresponding impose
compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

# print compensation
def compMean(start,stop,compIn,compOut):
    consol=pd.DataFrame(columns=['fly','gain','impose','compensation'])


    print "calculating Mean"
    #if there are more starts than stop because of abrupt ending while recording
    #remove the last start event
    try: 
        assert len(start)==len(stop)
    except AssertionError:
        print 'assertion error'
        start=start[:-1] 

    for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
        currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
        compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
        curr=pd.DataFrame([{'fly':parameters['fly'],
                                             "gain":df.trajectory__gain[start.index[i]],
                                             "impose":df.trajectory__impose[start.index[i]],
                                             'compensation':currCompensationMean}])
        consol=consol.append(curr,ignore_index=True)        
    cols=['fly','gain','impose','compensation']
    consol=consol[cols]
#         print "total measn were",i+1
    return consol

cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)



#     plt.plot(df.trajectory__imposeResponseSmooth)
#     plt.plot(compensation,c=(1,0,0,0.2))
#     plt.plot(df.trajectory__impose)
# #     plt.plot(cp,'g')
#     plt.axhline(y=1)
#     plt.title(winTitle)
#     plt.show()



In [ ]:
impose=df.trajectory__impose.copy()
impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
compensation=df.trajectory__imposeResponseSmooth/impose
# plt.plot(df.trajectory__imposeResponseSmooth)
plt.plot(compensation,c=(1,0,0,0.2))
plt.plot(df.trajectory__impose)
plt.plot(cp,'g')
plt.axhline(y=1)
plt.show()

In [ ]:
'''Load a dataframe from pickle'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*df.pickle"])
fullConsol=pd.DataFrame()
# print paths, "\n"
for path in paths:
    
    tic()
    print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
#     parameters=data["metadata"]['parameters']

    toc()



In [ ]:
plt.scatter(df.trajectory__position_x,df.trajectory__position_y)
plt.show()

In [ ]:
maxdH=1
step=100
tot=1000


initX=10
initY=10
initH=90

prevH=initH
prevX=initX
prevY=initY

fullX=np.array([initX])
fullY=np.array([initY])

for i in range(tot):
    dH=maxdH*np.random.rand(1)
    newH=prevH+dH
    
    newX=prevX+step*np.cos(np.degrees(newH))
    newY=prevY+step*np.sin(np.degrees(newH))
    
    fullX=np.append(fullX,newX)
    fullY=np.append(fullY,newY)
    
    prevX=newX
    prevY=newY


# print fullX,fullY
plt.scatter(fullX,fullY)
plt.show()

In [ ]:
parameters

In [ ]:
#flips are position where there is a rise or fall
flips=np.sign(np.diff(df.trajectory__impose))

#impose is impose df without first element
impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

#cw is clockWise, ccw is Counter clockwise
#start and stop are positions where the impose starts or stops
#start(stop) is zero everywhere except when there is the respective start(stop) event
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

# plt.plot(np.array(impose),'r')
# plt.plot(np.array(flips),'g')
# plt.plot(np.array(ccwStart)*1,'b')
# plt.plot(np.array(ccwStop)*1,'k')
# plt.plot(np.array(cwStart)*1,'m')
# plt.plot(np.array(cwStop)*1,'y')
# print ccwStart.shape,ccwStop.shape
# print cwStart.shape,cwStop.shape
plt.show()

#(c)cwStart(stop)Index is just those positions where flips occured

ccwStartIndex= ccwStart[ccwStart==1]
cwStartIndex= cwStart[cwStart==1]
ccwStopIndex= ccwStop[ccwStop==1]
cwStopIndex= cwStop[cwStop==1]
print ccwStartIndex.shape
print ccwStopIndex.shape

print cwStartIndex.shape
print cwStopIndex.shape


cp = pd.DataFrame(index=compensation.index,columns=['a'])

#compensationMean is a df who is zero when impose is false, 
# and has mean value of compensation during the corresponding impose
compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

# print compensation
def compMean(start,stop,compIn,compOut):
    print "calculating Mean"
    #if there are more starts than stop because of abrupt ending while recording
    #remove the last start event
    try: 
        assert len(start)==len(stop)
    except AssertionError:
        print 'assertion error'
        start=start[:-1] 

    for i in range(len(stop)):

        print "i mean is",compIn.ix[start.index[i]:
                                          stop.index[i]].mean()
        compOut.ix[start.index[i]:stop.index[i]]=compIn.ix[start.index[i]:stop.index[i]].mean()

    print "total measn were",i+1
compMean(cwStartIndex,cwStopIndex,compensation,cp)
compMean(ccwStartIndex,ccwStopIndex,compensation,cp)

In [ ]:
df.trajectory__imposeResponse.plot()
df.trajectory__impose.plot()

plt.show()

In [ ]:
plt.plot(pd.rolling_mean((df.trajectory__imposeResponse),window=10))
plt.plot(pd.rolling_mean((df.trajectory__imposeResponse),window=100),'r')
plt.show()
         

In [ ]:
# ccwStart=pd.DataFrame(0, index=np.arange(len(df)), columns=["feature_list"])
flips=np.sign(df.trajectory__impose.diff)
ccwStart[flips>0]=1
plt.plot(flips)
plt.show()

In [ ]:
flips=np.sign(np.diff(df.trajectory__impose))
impose=df.trajectory__impose.drop(df.trajectory__impose.index[1])
# a=np.roll(a,-1)
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

plt.plot(np.array(impose),'r')
plt.plot(np.array(flips),'g')
plt.plot(np.array(ccwStart)*1,'b')
plt.plot(np.array(ccwStop)*1,'k')
plt.plot(np.array(cwStart)*1,'m')
plt.plot(np.array(cwStop)*1,'y')
print ccwStart.shape,ccwStop.shape
print cwStart.shape,cwStop.shape

ccwStart= ccwStart[ccwStart==1]
cwStart= cwStart[cwStart==1]
ccwStop= ccwStop[ccwStop==1]
cwStop= cwStop[cwStop==1]
print ccwStart,ccwStop

plt.show()

In [ ]:
cp = pd.DataFrame(index=compensation.index,columns=['a'])
# print compensation
for i in range(len(ccwStart)):
    try: 
        assert len(ccwStart)==len(ccwStop)
    except AssertionError:
        ccwStop=ccwStop[:-1]
    
    print "i mean is",compensation.ix[cwStart.index[i]:
                                      cwStop.index[i]].mean()
    cp.ix[cwStart.index[i]:cwStop.index[i]]=compensation.ix[cwStart.index[i]:cwStop.index[i]].mean()
    

In [7]:
fullConsol=pd.DataFrame()
ccw,cw=analyser('a')
fullConsol=fullConsol.append(ccw,ignore_index=True)
fullConsol=fullConsol.append(cw,ignore_index=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=165,center=False).mean()


calculating Mean
assertion error
calculating Mean


In [8]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Panel, Tabs
output_file("line.html")

x=df.trajectory__impose.index
y1=df.trajectory__impose
y2=df.trajectory__imposeResponse
y3=df.trajectory__imposeResponseSmooth
y4=compensationSmooth
y5=cp.fillna(0).abs()
# y5=cp

p1 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p2 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p3 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p4 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p5 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")

p1.line(x,y1, line_width=2, line_color="navy")

p2.line(x,y2, line_width=0.75,line_color="orange",line_alpha=0.2)

p3.line(x,y1, line_width=2, line_color="navy")
p3.line(x,y2, line_width=0.75,line_color="orange",line_alpha=0.2)
p3.line(x,y3, line_width=1,line_color="olive",line_alpha=1)

p4.line(x,y1, line_width=2, line_color="navy")
p4.line(x,y3, line_width=0.75,line_color="orange",line_alpha=0.5)
p4.line(x,y4, line_width=1,line_color="olive",line_alpha=0.5)

p5.line(x,y5, line_width=2,line_color="darkgrey",line_alpha=1)
p5.line(x,y1, line_width=2, line_color="navy")
p5.line(x,y3, line_width=0.75,line_color="orange",line_alpha=0.5)
p5.line(x,y4, line_width=1,line_color="olive",line_alpha=0.5)


tab1 = Panel(child=p1, title="impose")
tab2 = Panel(child=p2, title="imposeResponse")
tab3 = Panel(child=p3, title="imposeResponseSmooth")
tab4 = Panel(child=p4, title="compensation")
tab5 = Panel(child=p5, title="compensationMean")




tabs = Tabs(tabs=[tab1,tab2,tab3,tab4 ,tab5])

show(tabs)


NameError: name 'compensationSmooth' is not defined

In [ ]:
# data['metadata']['parameters']['fly']
# print easygui.filesavebox()
parameters['fly']

In [ ]:
a={'a':1,'b':2,'c':3,'d':5}
b={'a':2,'b':2,'c':3,'d':5}

# d.append(df, ignore_index=True)
df=pd.DataFrame([a])
df=df.append(pd.DataFrame([b]))#,ignore_index=True)
print df
# print d

In [ ]:
a

In [ ]:
print (df.trajectory__impose.shape)
a.shape

In [ ]:
(.3195+.34179)/2

In [ ]:
plt.plot(flips)
plt.plot(np.array(b),'r')

plt.show()

In [ ]:
df.trajectory__wbad.plot.hist(bins=60)

In [ ]:
''' 
Load multiple dataframe pickle file into memory and append it
'''
paths=easygui.fileopenbox(multiple=True,filetypes=['*.pickle'])
i=0
a=range(len(paths))
for path in paths:
    a[i]=pd.read_pickle(path)

    i+=1
    
df=pd.concat(a)

In [ ]:
#load params pickle
path=easygui.fileopenbox()
import cPickle as pl
import json
a=pl.load(open( path, "rb" ))
params=a[1]

In [ ]:
parameters


In [ ]:
df.trajectory__trial.max()

In [ ]:
''' 
Load multiple bag files to be converted to dataframe
'''

paths=easygui.fileopenbox(multiple=True,filetypes=['*.bag'])
i=0
for path in paths:

    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    df.columns
    bag=rosbag.Bag(path)
    for topic,msg,t in bag.read_messages(topics='/metadata'):
        a=msg
    parameters=json.loads(a.data)
    picklepath=path+"_df.pickle"
    df.to_pickle(picklepath)
    i+=1
    
print "done"

In [ ]:
'''
plot a single df
'''

from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Slider
from bokeh.io import output_file, show, vform


slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")

# prepare some data
N = len(df.trajectory__position_x)
x = df.trajectory__position_x
y = df.trajectory__position_y
h=np.deg2rad(df.trajectory__orientation_x)

dec=1
x=x[::dec]
y=y[::dec]
h=h[::dec]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file(path+".html", title="Trajectory Quad", mode="cdn")

TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,undo,redo,save"

# create a new plot with the tools above, and explicit ranges
p = figure(tools=TOOLS, x_range=(0,256), y_range=(0,256),webgl=True,plot_width=1000, plot_height=1000)
p.image(image=[np.flipud(np.rot90(np.invert((parameters["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

# add a circle renderer with vectorized colors and sizes
# p.circle(x,y,size=4  ,fill_alpha=0.5, line_color=None)

print vform(slider)
p.triangle(x,y,size=slider ,angle=h ,fill_alpha=0.8, line_color=None)
# p.wedge(x, y, radius=0.2, start_angle=0.4, end_angle=4.8,
#         color="firebrick", alpha=0.6, direction="clock")
# show the results
show(p)

In [ ]:

from bokeh.models.widgets import Slider
from bokeh.io import output_file, show, vform

output_file("slider.html")

slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")

show(vform(slider))


import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.get('data');
    var A = amp.get('value')
    var k = freq.get('value')
    var phi = phase.get('value')
    var B = offset.get('value')
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.trigger('change');
""")

amp_slider = Slider(start=0.1, end=10, value=1, step=.1,
                    title="Amplitude", callback=callback)
callback.args["amp"] = amp_slider

freq_slider = Slider(start=0.1, end=10, value=1, step=.1,
                     title="Frequency", callback=callback)
callback.args["freq"] = freq_slider

phase_slider = Slider(start=0, end=6.4, value=0, step=.1,
                      title="Phase", callback=callback)
callback.args["phase"] = phase_slider

offset_slider = Slider(start=-5, end=5, value=0, step=.1,
                       title="Offset", callback=callback)
callback.args["offset"] = offset_slider

layout = row(
    plot,
    widgetbox(amp_slider, freq_slider, phase_slider, offset_slider),
)

output_file("slider.html", title="slider.py example")

show(layout)

In [ ]:
'''plotdoor field'''
p = figure(x_range=(0, 255), y_range=(0,255))

parameters["odourField"]=np.array(parameters["odourField"]['__ndarray__'])

p.image(image=[np.flipud(np.rot90(rescale(parameters['odourField'], 10)))], x=0, y=0, dw=255, dh=255, palette="Greys3")


# must give a vector of image data for image parameter

output_file("image.html", title="image.py example")

show(p)  # open a browser

In [ ]:
# parameters["odourField"]=parameters["odourField"]['__ndarray__']
parameters['odourField']=np.array((parameters["odourField"]))
from skimage.transform import rescale
rescale(parameters['odourField'], 10).shape


In [ ]:
#load df pickle
path=easygui.fileopenbox()
df=pd.read_pickle(path)
path=path.rstrip("_df.pickle")

In [ ]:
files=easygui.fileopenbox("select the files to be dataframed and plotted",multiple=True)

In [ ]:
'''
plot a single df
'''

from bokeh.plotting import figure, output_file, show, gridplot

# prepare some data
N = len(df.trajectory__position_x)
x = df.trajectory__position_x
y = df.trajectory__position_y
h=np.deg2rad(df.trajectory__orientation_x)

dec=6
x=x[::dec]
y=y[::dec]
h=h[::dec]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file(path+".html", title="Trajectory Quad", mode="cdn")

TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,undo,redo,save"

# create a new plot with the tools above, and explicit ranges
w=500
h=500 
s=3
bl = figure(tools=TOOLS, x_range=(64-32,64+32), y_range=(32-32,32+32),webgl=True,plot_width=w, plot_height=h)
tl = figure(tools=TOOLS, x_range=(64-32,64+32), y_range=(161-32,161+32),webgl=True,plot_width=w, plot_height=h)
br = figure(tools=TOOLS, x_range=(193-32,193+32), y_range=(32-32,32+32),webgl=True,plot_width=w, plot_height=h)
tr = figure(tools=TOOLS, x_range=(193-32,193+32), y_range=(161-32,161+32),webgl=True,plot_width=w, plot_height=h)

bl.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
tl.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
br.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
tr.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)

bl.image(image=[np.flipud(np.rot90(np.invert((params["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")


tl.image(image=[np.flipud(np.rot90(np.invert((params["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

br.image(image=[np.flipud(np.rot90(np.invert((params["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

tr.image(image=[np.flipud(np.rot90(np.invert((params["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

p = gridplot([[tl, tr], [bl, br]])

# add a circle renderer with vectorized colors and sizes
# p.circle(x,y,size=4  ,fill_alpha=0.5, line_color=None)
# p.triangle(x,y,size=4 ,angle=h ,fill_alpha=1, line_color=None)
# p.wedge(x, y, radius=0.2, start_angle=0.4, end_angle=4.8,
#         color="firebrick", alpha=0.6, direction="clock")
# show the results
show(p)

In [ ]:
params["odourField"]

In [ ]:
a=range(0,15)
a=a[::5]
a

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

N = 1000
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(x_range=(0, 10), y_range=(0, 10))

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, color_mapper=1)

output_file("image.html", title="image.py example")

show(p)  # open a browser

In [ ]:
from hcpWorldGen import WorldGen
df.plot.scatter('trajectory__position_x',y='trajectory__position_y',s=0.01)
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pl

# Plot simple sinus function
fig_handle = plt.figure()
x = df.trajectory__position_x
y = df.trajectory__position_y

h=np.deg2rad(df.trajectory__orientation_x)

dec=1

x=x[::dec]
y=y[::dec]
h=h[::dec]
plt.scatter(x,y,marker='o',s=5,color=(1,0,0,0.1802))

# plt.scatter(initPlot.odd[:,0],initPlot.odd[:,1],color=initPlot.oddPlotColor,marker=initPlot.oddPlotMarker)
# plt.scatter(initPlot.even[:,0],initPlot.even[:,1],color=initPlot.evenPlotColor,marker=initPlot.evenPlotMarker),marker='|',color='g')
# plt.axis([0,255,0,255])

plt.show()


# Save figure handle to disk
# pl.dump(fig_handle,file('sinus.pickle','w'))



In [ ]:
print type(fig_handle)

In [ ]:
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11")

In [ ]:
# parameters["odourField"]=np.array(parameters["odourField"]['__ndarray__'])
x = df.trajectory__position_x
y = df.trajectory__position_y
c=df.trajectory__valve
h=np.deg2rad(df.trajectory__orientation_x)

dec=3

x=x[::dec]
y=y[::dec]
h=h[::dec]
c=c[::dec]

plt.imshow(np.flipud(np.rot90(parameters["odourField"])),cmap='Greys')
plt.scatter(x,y,marker='o',s=5,c=c,cmap='seismic',lw=0)

plt.show()

In [ ]:
df.trajectory__valve.hist()
plt.show()

In [ ]:
'''plot hex bin'''

df.plot.hexbin(x='trajectory__position_x',y='trajectory__position_y',gridsize=100,cmap=plt.cm.viridis,)
plt.show()


In [ ]:
WorldGen.initPositions

In [ ]:
df.plot.hexbin(x='trajectory__position_x',y='trajectory__position_y',gridsize=100)

In [ ]:
plt.show()

In [ ]:
df.columns

In [ ]:
plt.imshow(np.flipud(np.rot90(parameters["odourField"])))
plt.show()

In [ ]:
params["odourField"].shape

In [ ]:
params

In [ ]:
a[1]

In [ ]:
2+3

In [ ]:
print paths

In [ ]:
# modules
#------------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as py
from matplotlib import animation

py.close('all') # close all previous plots

# create a random line to plot
#------------------------------------------------------------------------------

x = df.trajectory__position_x
y = df.trajectory__position_y
t = np.linspace(0,1,x.shape[0]) # your "time" variable
# py.figure(1)
# py.scatter(0, 0, s=1)
# py.axis([0, 255, 0, 255])
# py.show()

# animation of a scatter plot using x, y from above
#------------------------------------------------------------------------------

fig = py.figure(1)
ax = py.axes(xlim=(0, 255), ylim=(0,255))
scat = ax.scatter([], [], s=20,marker=(3,0,0))

def init():
    scat.set_offsets([])
    return scat,

def animate(i):
    data = np.hstack((x[:i,np.newaxis], y[:i, np.newaxis]))
    scat.set_offsets(data)
    scat.se
#     scat.set_marker('o')
    return scat,

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(x)+1, 
                               interval=1000/165., blit=True, repeat=False)
plt.show()

In [ ]:
len(df.trajectory__position_x)

In [ ]:
i=360


In [ ]:
import matplotlib


In [ ]:
print matplotlib.__version__